In [ ]:
.libPaths("~/R/x86_64-pc-linux-gnu-library/4.4/")

In [ ]:
library(SingleCellExperiment)

In [ ]:
# Loading libraries
library("Matrix")
library(Seurat)
library(DT)
library(dplyr)
library(here)
library(ggplot2)
library(kableExtra)
library(cowplot)
library(tidyverse)
library(reshape)
library(SingleR)
library(cowplot)
library(fgsea)
library(org.Mm.eg.db)
library(biomaRt)
library(furrr)
library(STACAS)

In [ ]:
ggtheme <- function() {
  theme(
    axis.text = element_text(size = 20, colour = "black"),
    axis.title = element_text(size = 20, colour = "black"),
    text = element_text(size = 20, colour = "black"),
    legend.text = element_text(size = 20, colour = "black"),
    legend.key.size =  unit(10, units = "points"),
    plot.background = element_blank()
     )
}

In [ ]:
load("datasets/NK.ver1.rda")

# Analysis of updated NK dataset

In [ ]:
DimPlot(NK, group.by = "harmony_clusters", reduction = "umap.harmony")

In [ ]:
NK  <- FindClusters(NK, resolution = 0.05)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4)
DimPlot(NK,reduction = "umap.harmony", label = T, label.size = 7)

In [ ]:
VlnPlot(NK, features = c("nFeature_RNA", "nCount_RNA"), pt.size = 0)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 4)
VlnPlot(NK, features = c("FCGR3A", "NCR1", "NCAM1","NKG7"), ncol = 4, pt.size = 0)

In [ ]:
mrk  <- FindAllMarkers(NK, only.pos = T)

In [ ]:
NK$Cohort  %>% table

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4)
DimPlot(NK,reduction = "umap.harmony", label = T, group.by = "Population", label.size = 7)

In [ ]:
NK$Population  %>% table

In [ ]:
T

In [ ]:
mrk  %>% dplyr::filter(cluster == 3)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 4)
VlnPlot(NK, features = c("C1R", "OGN", "MGP","ADIRF"), ncol = 4, pt.size = 0)

In [ ]:
mrk  %>% dplyr::filter(cluster == 4)

In [ ]:
FeaturePlot(NK, features = c("NCR1","CD3D","CCL5","TRAC"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(NK, features = c("NCAM1","FCER1G","NKG2D","CTLA4"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(NK, features = c("CD3D","CD3E","LAT","TRAC"), reduction = "umap.harmony")

## Separation of NK1 and NK2 populations

In [ ]:
nk1  <- subset(NK, seurat_clusters == 2)
nk2  <- subset(NK, seurat_clusters == 1)


In [ ]:
DimPlot(nk1, reduction = "umap.harmony")

In [ ]:
DimPlot(nk2, reduction = "umap.harmony")

In [ ]:
nk2$umap_x  <- nk2@reductions$umap.harmony@cell.embeddings[,1]

In [ ]:
nk2$umap_x_keep  <-   nk2$umap_x < -2.5

In [ ]:
DimPlot(nk2, reduction = "umap.harmony", group.by = "umap_x_keep")

In [ ]:
nk2  <- subset(nk2, umap_x_keep == TRUE)


In [ ]:
DimPlot(nk2, reduction = "umap.harmony")

In [ ]:
nk2  <- FindClusters(nk2, res = 0.5)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)
DimPlot(nk2, reduction = "umap.harmony", label = T, label.size = 7)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)
FeaturePlot(nk2, reduction = "umap.harmony", features = "NCAM1")

In [ ]:
FeaturePlot(nk2, reduction = "umap.harmony", features = "IL7R")

In [ ]:
FeaturePlot(nk2, reduction = "umap.harmony", features = "GNLY")

In [ ]:
FeaturePlot(nk2, reduction = "umap.harmony", features = "CTLA4")

In [ ]:
FeaturePlot(nk2, reduction = "umap.harmony", features = "PDCD1")

In [ ]:
FeaturePlot(nk2, reduction = "umap.harmony", features = "LAG3")

In [ ]:
FeaturePlot(nk2, reduction = "umap.harmony", features = "KLRC1")

In [ ]:
FeaturePlot(nk2, reduction = "umap.harmony", features = "IFI44L")

In [ ]:
FeaturePlot(nk2, reduction = "umap.harmony", features = "NFKBIA")

## Markers of NK2 clusters

In [ ]:
mrk  <- FindAllMarkers(nk2)

In [ ]:
mrk

In [ ]:
mrk  %>% dplyr::filter(avg_log2FC > 0)  %>% 
group_by(cluster)  %>% slice_head(n = 10)  %>% pull(gene)  %>% unique

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 15)
DotPlot(nk2, features = mrk  %>%  dplyr::filter(avg_log2FC > 0) %>% 
        group_by(cluster)  %>% slice_head(n = 10)  %>% pull(gene)  %>% unique) +
coord_flip()

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 10)
DotPlot(nk2, features = c("CST7","CCL5","KLRB1","GZMM","CD160","GZMA","CD247",
                          "KLRC1","NFKBIA","REL","NFKBIZ","NFKB1","ICAM1",
                         "SELL","IL7R","CCR7","ITGAX","GNLY","TNFRSF11A",
                         "ITGA1","RBPJ","NCAM1","CSF1","IFI44L","PDCD1","CTLA4","LAG3",
                         "DUSP1","CCL3","CCL4","FOS","FOSB")) +
coord_flip() + scale_color_gradient2(low = "dodgerblue",mid = "white", high = "indianred2") +
scale_size_continuous(range = c(2,5)) +
xlab("") +
theme(axis.text.x = element_text(angle = 00)) 

In [ ]:
mrk  %>% dplyr::filter(cluster == 0 & avg_log2FC>0)

# Slingshot trajectory in NK2 cells

In [ ]:
.libPaths("~/R/x86_64-pc-linux-gnu-library/4.4/")


library(grDevices)
library(RColorBrewer)
library(slingshot)

In [ ]:
saveRDS(nk2, "datasets/nk2.rds")

In [ ]:
nk2  <- readRDS("datasets/nk2.rds")

In [ ]:
DimPlot(nk2, reduction = "umap.harmony")

In [ ]:
nk2_without_dead  <- subset(nk2, seurat_clusters %in% c(0:4))

In [ ]:
nk2  <- nk2_without_dead

In [ ]:
nk2

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)
DimPlot(nk2, reduction = "umap.harmony")

In [ ]:
mtx  <- nk2@assays$RNA@layers$scale.data

In [ ]:
rownames(mtx)  <- rownames(nk2@assays$RNA@layers$scale.data)

In [ ]:
scaled_matrix <- GetAssayData(object = nk2, slot = "scale.data")

In [ ]:
scaled_matrix

In [ ]:
write.csv(scaled_matrix, "tables/nk_scale_data_matrix.csv")

In [ ]:
nk2@assays$RNA@layers$scale.data

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)
DimPlot(nk2, reduction = "umap.harmony")

In [ ]:
nk2

In [ ]:
p <- DotPlot(nk2, features = c("CST7","CCL5","KLRB1","GZMM","CD160","GZMA","CD247",
                          "KLRC1","NFKBIA","REL","NFKBIZ","NFKB1","ICAM1",
                         "SELL","IL7R","CCR7","ITGAX","GNLY","TNFRSF11A",
                         "ITGA1","RBPJ","NCAM1","CSF1","IFI44L","PDCD1","CTLA4","LAG3",
                         "DUSP1","CCL3","CCL4","FOS","FOSB"))
# Extract source data:
dotplot_data <- p$data
head(dotplot_data)

In [ ]:
write.csv(dotplot_data, "tables/251009_source_data_2G.csv")

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
DotPlot(nk2, features = c("CST7","CCL5","KLRB1","GZMM","CD160","GZMA","CD247",
                          "KLRC1","NFKBIA","REL","NFKBIZ","NFKB1","ICAM1",
                         "SELL","IL7R","CCR7","ITGAX","GNLY","TNFRSF11A",
                         "ITGA1","RBPJ","NCAM1","CSF1","IFI44L","PDCD1","CTLA4","LAG3",
                         "DUSP1","CCL3","CCL4","FOS","FOSB")) +
coord_flip() + scale_color_gradient2(low = "dodgerblue",mid = "white", high = "indianred2") +
scale_size_continuous(range = c(2,5)) +
xlab("") +
theme(axis.text.x = element_text(angle = 00),
     axis.text.y = element_text(face = "italic"))  +
ggtheme()
#ggsave("figures/nk_populations_dot_plot.svg", width = 15, height = 22, units = "cm")

In [ ]:
DotPlot(nk2, features = c("CST7","CCL5","KLRB1","GZMM","CD160","GZMA","CD247",
                          "KLRC1","NFKBIA","REL","NFKBIZ","NFKB1","ICAM1",
                         "SELL","IL7R","CCR7","ITGAX","GNLY","TNFRSF11A",
                         "ITGA1","RBPJ","NCAM1","CSF1","IFI44L","PDCD1","CTLA4","LAG3",
                         "DUSP1","CCL3","CCL4","FOS","FOSB"))

In [ ]:
DotPlot(nk2, features = c("CST7","CCL5","KLRB1","GZMM","CD160","GZMA","CD247",
                          "KLRC1","NFKBIA","REL","NFKBIZ","NFKB1","ICAM1",
                         "SELL","IL7R","CCR7","ITGAX","GNLY","TNFRSF11A",
                         "ITGA1","RBPJ","NCAM1","CSF1","IFI44L","PDCD1","CTLA4","LAG3",
                         "DUSP1","CCL3","CCL4","FOS","FOSB")) +
coord_flip() + scale_color_gradient2(low = "dodgerblue",mid = "white", high = "indianred2") +
scale_size_continuous(range = c(2,5)) +
xlab("") +
theme(axis.text.x = element_text(angle = 00),
     axis.text.y = element_text(face = "italic"))  

In [ ]:
DefaultAssay(nk2) <- "RNA"

fun_basic <- nk2

Idents(fun_basic) <- fun_basic$seurat_clusters

sce <- as.SingleCellExperiment(fun_basic)

reducedDim(sce, "PCA", withDimnames=TRUE) <- nk2[['pca']]@cell.embeddings
#reducedDim(sce, "UMAP", withDimnames=TRUE) <- fun_basic[['umap']]@cell.embeddings
reducedDim(sce, "UMAP.harmony", withDimnames=TRUE) <- fun_basic[['umap.harmony']]@cell.embeddings

In [ ]:
sce <- slingshot(sce, clusterLabels = 'seurat_clusters', reducedDim = 'UMAP.harmony', start.clus= 2)

In [ ]:
colors <- colorRampPalette(brewer.pal(11,'Spectral')[-6])(100)
plotcol <- colors[cut(sce$slingPseudotime_1, breaks=100)]

In [ ]:
sce$slingPseudotime_1  <- sce$slingPseudotime_1*-1

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 4)
plot(reducedDims(sce)$UMAP.harmony, col = colors[cut(sce$slingPseudotime_1, breaks=100)], pch = 16)

In [ ]:
md  <- data.frame(sce@colData$slingPseudotime_1)

In [ ]:
md$slingPseudotime  <- sce@colData$slingPseudotime_1

In [ ]:
plot(reducedDims(sce)$UMAP.harmony, col = colors[cut(md$slingPseudotime, breaks=100)], pch = 16)
lines(SlingshotDataSet(sce), lwd=4, col='black')

In [ ]:
md  <- data.frame(sce@colData$slingPseudotime_1)

In [ ]:
coord  <- as.data.frame(nk2@reductions$umap.harmony@cell.embeddings)

In [ ]:
coord$seurat_clusters  <-  nk2$seurat_clusters

In [ ]:
write.csv2(coord, "tables/251009_source_data_2F.csv")

In [ ]:
write.csv2(coord, "tables/251009_source_data_2F.csv")

In [ ]:
coord  <- as.data.frame(nk2@reductions$umap.harmony@cell.embeddings)

In [ ]:
coord$pseudotime  <- sce$slingPseudotime_1

In [ ]:
coord

In [ ]:
write.csv2(coord, "tables/251009_source_data_2H.csv")

In [ ]:
pdf(file = "figures/revisions_trajectories.pdf", width = 6, height = 6)
plot(reducedDims(sce)$UMAP.harmony, col = colors[cut(sce$slingPseudotime_1, breaks=100)], pch = 16)
lines(SlingshotDataSet(sce), lwd=4, col='black')
dev.off()

In [ ]:
nk2$Cohort  %>% table

In [ ]:
psts <- slingPseudotime(sce) %>%
  as.data.frame() %>%
  mutate(cells = rownames(.),
         conditions = nk2$Cohort) %>%
  pivot_longer(starts_with("Lineage"), values_to = "pseudotime", names_to = "lineages")

In [ ]:
psts

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 4)
psts  %>% 
dplyr::filter(lineages == "Lineage1")  %>% 
ggplot(aes(x = pseudotime, color = conditions, fill = conditions)) +
  geom_density(alpha = .03) +
  #scale_fill_brewer(type = "qual") +
  facet_wrap(~lineages) +
  theme(legend.position = "bottom") +
theme_classic() +
ggtheme()

ggsave("figures/nk_populations_pseudotime.svg", width = 24, height = 8, units = "cm")

# Source data

In [ ]:
int_nk  <- readRDS("datasets/int_nk.rds")

In [ ]:
int_nk$nk_type  %>% table

In [ ]:
avgexp = AggregateExpression(int_nk, 
                             return.seurat = F, group.by = c("nk_type", "sample"),
                          assay = "SCT", slot = "scale.data")

In [ ]:
rownames(avgexp$SCT)  <- rownames(int_nk)

In [ ]:
genes  <- c('CLEC2D', 'CRTAM', 'HAVCR2', 'KLRC1', 'KLRC2', 'NCAM1', 'NR4A1', 
'NR4A3', 'ZEB2', 'DNAJB1','FCGR3A', 'HSPA1A','KIR2DL3','KLRB1', 'KLRD1', 
'KLRF1', 'KLRK1', 'NCR3', 'FCGR3A', 
'FGFBP2','GZMA', 'GZMB', 'GZMH', 'IFNG', "GZMK", "IL12B",
'PRF1', 'TNFSF10', "IFNGR1", "TBX21", "CST7", "S100B","CCL3","CCL4")

avgexp2  <- avgexp$SCT[which(rownames(avgexp$SCT) %in% genes),]

In [ ]:
avgexp2

In [ ]:
avgexp_nsclc_2_a  <- avgexp2

In [ ]:
avgexp_nsclc_2_a

In [ ]:
library(pheatmap)

options(repr.plot.width = 4, repr.plot.height = 9)
pheatmap(avgexp_nsclc_2_a, main = "", scale = "row", cluster_cols = F, cluster_rows = T,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 9, height = 3.3,
                  fontsize = 9)

In [ ]:
genes  <- c( 'HAVCR2', 'KLRC1',  'NCAM1', 
'KLRF1', 'KLRK1', 'NCR3', 'FCGR3A',  "PRF1",
'FGFBP2', 'GZMB', 'GZMH', 'IFNG', "GZMK", 
'PRF1', 'TNFSF10', "TBX21", "CST7", "NKG7", 
           "CX3CR1", "CD44","TOX","CTLA4")

avgexp3  <- avgexp$SCT[which(rownames(avgexp$SCT) %in% genes),]

options(repr.plot.width = 4, repr.plot.height = 6)
pheatmap(avgexp3, main = "", scale = "row", cluster_cols = F, cluster_rows = T,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 9, height = 3.3,
                  fontsize = 9)

In [ ]:
avgexp3  %>% as.data.frame()